In [ ]:
import os    
os.environ['THEANO_FLAGS'] = "device=gpu1"    
import theano

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import keras
import matplotlib.pyplot as plt
import pickle

In [ ]:
class History(keras.callbacks.Callback):
    def __init__(self,):
        self.history = {'loss':[], 'val_loss':[], 'fbeta_score':[], 'val_fbeta_score':[], 'acc':[], 'val_acc':[]}

In [ ]:
dataset_index = 4

In [ ]:
model = ResNet50(include_top=False, weights='imagenet', input_shape=(3, 224, 224))

In [ ]:
#model.summary()

In [ ]:
model.layers[-1].outbound_nodes = []
model.outputs = [model.layers[-1].output]
output = model.get_layer('avg_pool').output
output = Flatten()(output)
output = Dense(output_dim=128, activation='relu')(output) # your newlayer Dense(...)
output = Dropout(0.5)(output)
output = Dense(output_dim=1,   activation='sigmoid')(output)
new_model = Model(model.input, output)

In [ ]:
#new_model.summary()

In [ ]:
for layer in new_model.layers[:-4]:
    layer.trainable = False

In [ ]:
model = new_model

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
              metrics=['fbeta_score', 'accuracy'])

In [ ]:
batch_size = 64
nb_classes = 2
nb_epoch = 50#change to 50 afterwards
nb_eval = 4#change to 4 afterwards
data_augmentation = True

In [ ]:
# input image dimensions
img_rows, img_cols = 224, 224
# The CIFAR10 images are RGB.
img_channels = 3

In [ ]:
import retina_dataset
# The data, shuffled and split between train and test sets:
(X_train, Y_train), (X_test, Y_test) = retina_dataset.load_data(dataset_index, 224)
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [ ]:
hist = History() 
for _ in range(nb_eval):
    print('Eval: '+str(_))
    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    new_hist = model.fit_generator(datagen.flow(X_train, Y_train ,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))
    hist.history['loss'] += new_hist.history['loss']
    hist.history['val_loss'] += new_hist.history['val_loss']
    hist.history['fbeta_score'] += new_hist.history['fbeta_score']
    hist.history['val_fbeta_score'] += new_hist.history['val_fbeta_score']
    hist.history['acc'] += new_hist.history['acc']
    hist.history['val_acc'] += new_hist.history['val_acc']

    model.save('models/resnet_pretrained_for_dataset'+str(dataset_index)+'_lr=-3_dense=128.h5')
    model.save_weights('models/resnet_pretrained_weights_for_dataset'+str(dataset_index)+'_lr=-3_dense=128.h5')
    plt.plot(hist.history['fbeta_score'])
    plt.plot(hist.history['val_fbeta_score'])
    plt.title('model f1 score')
    plt.ylabel('f1')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    with open('models/resnet_pretrained_for_dataset'+str(dataset_index)+'_lr=-3_dense=128_trainning_history', 'w') as f:  # Python 3: open(..., 'wb')
        pickle.dump(hist, f)

In [ ]:
import pickle
with open('models/resnet_pretrained_for_dataset'+str(dataset_index)+'_lr=-3_dense=128_trainning_history', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump(hist, f)